The largest eigen vector of a stochastic matrix

In [1]:
import csv
from collections import defaultdict

pairlist=[]
connections=defaultdict(list)
userset=set()

with open('linkedin_lex.csv', 'rb') as csvfile:
    allrows = csv.reader(csvfile, delimiter=',')
    for row in allrows:
#        if ((row[0]=='your_name') | (row[1]=='your_name')): continue   # exclude yourself ?
        pairlist.append((row[0], row[1]))
        connections[row[0]].append(row[1])
        connections[row[1]].append(row[0])
        userset.add(row[0])
        userset.add(row[1])
        
## Actual algorithm starts here        
        
pagerank=defaultdict(lambda:1./len(userset))  # to get normalized values (e.g., to compare with eigenvectors) = probabilities
#pagerank=defaultdict(lambda:1.)  

for iteration in xrange(0, 30):
    newpagerank=defaultdict(lambda:0.)
    for user in userset:
        for connection in connections[user]:
            newpagerank[user] += pagerank[connection]/len(connections[connection])
    pagerank = newpagerank


prs = sorted(pagerank.iteritems(), reverse=True, key=lambda (k,v): v)
print prs[:10]

[('your_name', 0.06708936740281303), ('marc_streit', 0.027623669072077136), ('joaquim_jorge', 0.024861691383864654), ('daniel_wagner', 0.017359336412605456), ('raghu_machiraju', 0.016577461872446186), ('hanspeter_pfister', 0.015788926933634907), ('lukas_gruber', 0.015780668882535095), ('dieter_schmalstieg', 0.01538867360176131), ('silvia_miksch', 0.014603463676250525), ('nils_gehlenborg', 0.014209074397919734)]


In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib
import math

g = nx.Graph()

remove_me = False

for user in userset:
    if remove_me & (user=='your_name'): continue
    g.add_node(user)
 
for user in userset:   
    if remove_me & (user=='your_name'): continue
    nconnec = 0
    for connection in connections[user]:
        if remove_me & (connection=='your_name'): continue
        g.add_edge(user, connection, weight = 1)
        nconnec+=1
    if remove_me & (nconnec==0):
        g.remove_node(user)
    
        
pagerank_nx = nx.pagerank_scipy(g)        
        
color = [(min(pagerank_nx[n]*30.,1),min(pagerank_nx[n]*30.,1), min(pagerank_nx[n]*30.,1)) for n in pagerank_nx]
pos = nx.spring_layout(g,  iterations=100)
nx.draw_networkx_edges(g, pos, width=1, alpha=0.4)
nx.draw_networkx_nodes(g, pos, node_color=color, node_size=100, alpha=1, linewidths =0.5)
#lbls = nx.draw_networkx_labels(g, pos) 

plt.show()

/Users/sen.li/anaconda/lib/python2.7/site-packages/matplotlib/collections.py:650: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors_original != str('face'):
/Users/sen.li/anaconda/lib/python2.7/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [3]:
# checks whether we have the same, or similar, pageranks
sorted_pr = sorted(pagerank_nx.iteritems(), reverse=True, key=lambda (k,v): v)
print sorted_pr[:10]

[('your_name', 0.06690176238997488), ('marc_streit', 0.025361622337392715), ('joaquim_jorge', 0.02114734605205533), ('nils_gehlenborg', 0.015273169521147954), ('raghu_machiraju', 0.014902764524823366), ('daniel_wagner', 0.014802371832301892), ('hanspeter_pfister', 0.014783200255599175), ('lukas_gruber', 0.013624181556571554), ('dieter_schmalstieg', 0.013335093882899982), ('silvia_miksch', 0.012799665300243618)]


In [5]:
# your number of connection
print 'my degree is: ', g.degree('your_name'), '\n'

# diameter = maximum nb of edges between 2 nodes = always 2 in this case
print 'the graph diameter is: ',nx.diameter(g), '\n'

#center : surprising ?
print 'the center is: ',nx.center(g), '\n'

# number  of clique communities  of 5 nodes
print 'there are ', len(list(nx.k_clique_communities(g, 5))),'clique communities\n'


# most influential ?
print 'degree: ', g.degree(sorted_pr[2]),'\n'


 my degree is:  170 

the graph diameter is:  4 

the center is:  ['your_name', 'marc_streit'] 

there are  2 clique communities

degree:  {'joaquim_jorge': 63} 

